In [ ]:
import numpy as np
import matplotlib.pyplot as plt 
from tqdm.auto import tqdm as progressbar
%matplotlib qt5

In [ ]:
import subprocess
access_str = r"pi@ender3.local"

def ssh_command(cmd):
    return subprocess.check_output([
            "ssh",
            access_str,
            cmd
        ]).decode()

print(ssh_command("v4l2-ctl --all").split("Flags:")[-1])
print(ssh_command("v4l2-ctl -cexposure_auto=1")[-500:])

In [ ]:
def set_exposure(value):
    return ssh_command(f"v4l2-ctl -cexposure_absolute={value}")

def set_temp(value):
    return ssh_command(f"v4l2-ctl -cwhite_balance_temperature={value}")

def set_gain(value):
    return ssh_command(f"v4l2-ctl -cset_gain={value}")

set_exposure(1)
ssh_command(f"v4l2-ctl -csaturation=32")

In [ ]:
import sys
import time
sys.path.append(r"C:\Users\Pedro\Desktop\TCC\ender-laser-scanner\ender")
from snapshot import MjpgSnapshotter


snap = MjpgSnapshotter("http://192.168.0.15/webcam/")

In [ ]:
images=[]
exposure_range = range(1,200,5)
for i in progressbar(exposure_range):
    set_exposure(i)
    ret, im = snap.read()
    time.sleep(0.5)
    if ret:
        images.append(im)

In [ ]:
fig, axs = plt.subplots(2)
axs[0].imshow(images[0])
axs[1].imshow(images[1])

In [ ]:
red = np.array([ [ [ [1],[-0.5],[-0.5] ] ] ])
white = np.array([ [ [ [1],[1],[1] ] ] ])
white=white/np.linalg.norm(white)

In [ ]:
I = np.stack(images)
I = I / 255 / (3**(1/3))
plt.figure()
red_contrast = I@red
white_value = I@white
plt.figure()
plt.plot(np.max(red_contrast,axis=(1,2)))
plt.plot(np.max(white_value, axis=(1,2)) )
plt.show()
print(np.argmax(np.max(red_contrast, axis=(1,2)) ))

In [ ]:
del ids
ids = []
print(I.shape)
for img in np.linalg.norm(I, axis=-1):
    ids.append( np.unravel_index(img.flatten().argmax(), img.shape) )
print(ids)
max_norm_vecs = [ I[n, i[0], i[1]] for n, i in enumerate(ids) ]

for m in max_norm_vecs:
    print(np.round(m,2)*(3*(1/3)))


In [ ]:
x = np.stack(max_norm_vecs) * 255 * 3**(1/3)
plt.plot(exposure_range, x[:,0], label="r", color ="red")
plt.plot(exposure_range, x[:,1], label="g", color ="green")
plt.plot(exposure_range, x[:,2], label="b", color ="blue")
plt.xlabel("exposure")
plt.legend()

In [ ]:
import pandas as pd

x = pd.Series(max_norm_vecs)
x.apply(lambda y: np.round(y,2)).to_csv("x.csv")

In [ ]:
plt.imshow(images[4])
plt.figure()
plt.imshow(images[10])

In [ ]:

plt.plot(im)